In [1]:
%cd ../../

/mnt/sda/otani/Experiments/eval-detection


In [3]:
import csv
import os
import numpy as np

from mim.utils import DEFAULT_CACHE_DIR
import os
import mmcv
from mmcv.utils.config import Config
import numpy as np
import pandas as pd
from src.extensions.metrics.ot_cost import get_ot_cost, get_cmap
from src.utils import krippendorff

from src.extensions.dataset.coco_custom import CocoOtcDataset
from tqdm import tqdm
import matplotlib.pyplot as plt
from shutil import copyfile

from concurrent.futures import ProcessPoolExecutor, as_completed
%matplotlib inline

In [26]:
ab2idx = {"A":0, "B": 1}
data = {f"user_{i+1}": [] for i in range(3)}
for k in data.keys():
    base_dir = f"data/raw/user_study/{k}/"
    files = os.listdir(base_dir)
    files.sort()
    for j in np.arange(1, 1100, 100):
        fn = f"{j}-{j+99}.csv"
        print(fn)
        data[k] += [ab2idx.get(row[-1], np.nan) for row in csv.reader(open(f"{base_dir}/{fn}"))]


1-100.csv
101-200.csv
201-300.csv
301-400.csv
401-500.csv
501-600.csv
601-700.csv
701-800.csv
801-900.csv
901-1000.csv
1001-1100.csv
1-100.csv
101-200.csv
201-300.csv
301-400.csv
401-500.csv
501-600.csv
601-700.csv
701-800.csv
801-900.csv
901-1000.csv
1001-1100.csv
1-100.csv
101-200.csv
201-300.csv
301-400.csv
401-500.csv
501-600.csv
601-700.csv
701-800.csv
801-900.csv
901-1000.csv
1001-1100.csv


In [27]:
# check missing data
for k, v in data.items():
    for i, c in enumerate(data[k]):
        if c is np.nan:
            print(k, i)

user_3 767


In [28]:
reliability_data = [v for v in data.values()]
krippendorff.alpha(reliability_data=reliability_data, level_of_measurement="nominal")

0.3572125944841349

In [35]:
np.isnan(selected)

True

In [45]:
input_data = [row[:2] for row in csv.reader(open("data/raw/user_study/inputs.csv"))]
selected_sys = {k: [] for k in data.keys()}
for k, u_data in data.items():
    for detectors, selected in zip(input_data, u_data):
        
        if np.isnan(selected):
            c = np.nan
        else:
            c = detectors[selected]
        selected_sys[k].append(c)

n = len(input_data)
n_disagree = 0
for i in range(n):
    n_yolof = [v[i] for v in selected_sys.values()].count("yolof")
    if (n_yolof!=0) and (n_yolof!=len(data)):
        n_disagree += 1

print(f"n-disagree {n_disagree}/{n} ({n_disagree/n})")

ref_is_yolof_better = []
for i in range(n):
    n_yolof = [v[i] for v in selected_sys.values()].count("yolof")
    n_retinanet = [v[i] for v in selected_sys.values()].count("retinanet")
    ref_is_yolof_better.append(n_yolof > n_retinanet)

print(ref_is_yolof_better[:15])


n-disagree 510/1057 (0.4824976348155156)
[True, True, True, True, True, False, True, True, True, True, True, True, True, True, True]


In [41]:
retina_res = mmcv.load("outputs/otc_search_param_candidates/20211027_003051/RetinaNet.pkl")
yolof_res = mmcv.load("outputs/otc_search_param_candidates/20211027_003051/YOLOF.pkl")
model_cfg = "retinanet_r50_fpn_2x_coco"
cfg = cfg = Config.fromfile(os.path.join(DEFAULT_CACHE_DIR, model_cfg + ".py"))
data_cfg = cfg.data
dataset = CocoOtcDataset(
    "data/coco/annotations/instances_val2017.json",
    data_cfg.test.pipeline,
    test_mode=True,
)

loading annotations into memory...
Done (t=0.57s)
creating index...
index created!


In [42]:
img_ids = os.listdir("data/processed/user_study/YOLOF")
img_ids.sort()
selected_indexes = [dataset.img_ids.index(int(x[:-4])) for x in img_ids]
retina_res = [retina_res[i] for i in selected_indexes]
yolof_res = [yolof_res[i] for i in selected_indexes]
gts = dataset.get_gts()
gts = [gts[i] for i in selected_indexes]

In [46]:
def get_accuracy(selected_sys, cost_a, cost_b):
    agree_counts = 0
    n = len(cost_a)
    for i in range(n):
        c = "retinanet" if cost_a[i] < cost_b[i] else "yolof"
        agree_counts += np.mean([v[i]==c for v in selected_sys.values()])
    return agree_counts

        
best_acc = 0
best_param = {}

step = 0.2
for p in np.arange(0.1, 0.99, step):
    for q in np.arange(0.1, 0.99, step):
        cmap_func = lambda x, y: get_cmap(
                    x, y, alpha=p, beta=q, mode="giou", use_dummy=True
                )
        retina_cc = [get_ot_cost(x, y, cmap_func) for x, y in zip(gts, retina_res)]
        yolof_cc = [get_ot_cost(x, y, cmap_func) for x, y in zip(gts, yolof_res)]
        is_yolof_better = [x > y for x, y in zip(retina_cc, yolof_cc)]

        acc = np.mean([x == y for x, y in zip(is_yolof_better, ref_is_yolof_better)])
        
        print(f"alpha={p:.2}, beta={q:.2}: acc={acc:.3}")
        if acc > best_acc:
            best_acc = acc
            best_param = {"alpha": p, "beta": q}

alpha=0.1, beta=0.1: acc=0.299
alpha=0.1, beta=0.3: acc=0.36
alpha=0.1, beta=0.5: acc=0.694
alpha=0.1, beta=0.7: acc=0.734
alpha=0.1, beta=0.9: acc=0.709
alpha=0.3, beta=0.1: acc=0.29
alpha=0.3, beta=0.3: acc=0.466
alpha=0.3, beta=0.5: acc=0.732
alpha=0.3, beta=0.7: acc=0.729
alpha=0.3, beta=0.9: acc=0.708
alpha=0.5, beta=0.1: acc=0.298
alpha=0.5, beta=0.3: acc=0.585
alpha=0.5, beta=0.5: acc=0.739
alpha=0.5, beta=0.7: acc=0.728
alpha=0.5, beta=0.9: acc=0.713
alpha=0.7, beta=0.1: acc=0.331
alpha=0.7, beta=0.3: acc=0.665
alpha=0.7, beta=0.5: acc=0.732
alpha=0.7, beta=0.7: acc=0.729
alpha=0.7, beta=0.9: acc=0.712
alpha=0.9, beta=0.1: acc=0.437
alpha=0.9, beta=0.3: acc=0.664
alpha=0.9, beta=0.5: acc=0.72
alpha=0.9, beta=0.7: acc=0.724
alpha=0.9, beta=0.9: acc=0.712


In [48]:
best_acc, best_param

(0.738883632923368, {'alpha': 0.5000000000000001, 'beta': 0.5000000000000001})